In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
pp = dict(surface='white', hemi='both')

# Get Forward Model

In [2]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.2s remaining:    2.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished


# Get sample data

In [3]:
settings = dict(number_of_sources=3, extents=(25, 40), duration_of_trial=1, target_snr=25)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\Lukas\Envs\invertenv\lib\site-packages\esinet\simulation.py:387: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_info = self.simulation_info.append(d, ignore_index=True)
 50%|█████     | 1/2 [00:01<00:01,  1.11s/it]c:\Users\Lukas\Envs\invertenv\lib\site-packages\esinet\simulation.py:387: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_info = self.simulation_info.append(d, ignore_index=True)
100%|██████████| 2/2 [00:00<00:00, 250.98it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00,  4.32it/s]


Using pyvistaqt 3d backend.

Using control points [4.35806489e-10 8.88959675e-10 5.52824216e-09]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Regularization Optimizations

In [9]:
# from invert.evaluate import corr, nmse
# from invert.solvers.lucas import SolverLUCAS

# solver = SolverLUCAS()
# solver.make_inverse_operator(fwd, verbose=0)
# solver.optimize_weights(fwd, info)
# solver.plot_weights()
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))

print("Mean correlation: ", np.mean(corr(stc_hat.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat.data, stc.data)))

Using control points [2.33299735e-10 2.84343006e-10 7.44851738e-10]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Mean correlation:  0.47291749281289114
Mean NMSE:  0.04191660047184385


In [5]:
from invert import Solver
from invert.evaluate import corr, nmse
solver_ = Solver(solver="laura")

solver_.make_inverse_operator(fwd, alpha="auto")
stc_hat = solver_.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))
print("Mean correlation: ", np.mean(corr(stc_hat.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat.data, stc.data)))

Using control points [1.18211566e-10 1.39541663e-10 3.55255038e-10]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Mean correlation:  0.21952747481307802
Mean NMSE:  0.045855399878377855


# Adapt

In [8]:
from invert.adapters import contextualize_bd, contextualize

stc_hat_cbd = contextualize_bd(stc_hat, fwd, lstm_look_back=10, num_units=128, num_epochs=50, verbose=0)
stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver.name))

print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))

Using control points [5.65340059e-11 8.59154754e-11 4.53238257e-10]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Mean correlation:  0.41758826870423504
Mean NMSE:  0.016038552466852325


# Evaluation

In [12]:
from invert import Solver
from invert.config import all_solvers

settings = dict(number_of_sources=(1,10), extents=(1, 40), duration_of_trial=1, target_snr=(1,25))


errors = {sname: [] for sname in all_solvers}
solvers = dict()
for i in range(25):
    print(i)
    sim = Simulation(fwd, info, settings).simulate(2)
    stc = sim.source_data[0]
    evoked = sim.eeg_data[0].average()

    for solver_name in all_solvers:
        solver = Solver(solver=solver_name)

        if solver.name.lower() == "multiple sparse priors" or "bayesian" in solver.name.lower():
            solvers[solver_name] = solver.make_inverse_operator(fwd, evoked, alpha="auto")
        else:
            if not solver_name in solvers:
                solvers[solver_name] = solver.make_inverse_operator(fwd, alpha="auto")
        stc_hat = solvers[solver_name].apply_inverse_operator(evoked)
        # stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))
        error = np.mean([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
        errors[solver_name].append( error )

0
-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 250.68it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00, 11.02it/s]


alpha must be set to a float when using Dynamic Statistical Parametric Mapping, auto does not work yet.
1
-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


...

100%|██████████| 2/2 [00:00<00:00,  9.68it/s]


24
-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 286.43it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00, 12.01it/s]


In [10]:
import pandas as pd
import seaborn as sns
sns.set(font_scale=0.8)
df = pd.DataFrame(errors)
sorted_index = df.median().sort_values().index
df = df[sorted_index]

plt.figure()
sns.boxplot(data=df)
plt.title("Correlation with ground truth")


df_mean_var = pd.concat([df.mean(), df.std()], axis=1)
df_mean_var = df_mean_var.rename(columns={0: "Median", 1: "Variance"})
df_mean_var["MedVar"] = df_mean_var["Median"] / df_mean_var["Variance"]
df_mean_var["Method"] = df_mean_var.index
display(df_mean_var)

plt.figure()
sns.scatterplot(x="Median", y="Variance", hue="Method", size="MedVar", data=df_mean_var)
plt.xlabel("Median")
plt.ylabel("Variance")


NameError: name 'errors' is not defined

In [ ]:
from invert.solvers.multiple_sparse_priors import SolverMultipleSparsePriors
from invert.solvers.loreta import SolverLORETA, SolverSLORETA, SolverELORETA
from invert.solvers.wrop import SolverBackusGilbert, SolverLAURA
from invert.solvers.smap import SolverSMAP
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm
solvers = [SolverMultipleSparsePriors, SolverLORETA, SolverSLORETA, SolverELORETA, SolverBackusGilbert, SolverLAURA, SolverSMAP, SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm]

for solver in solvers:
    solver_ = solver()
    if solver_.name == "Multiple Sparse Priors":
        solver_.make_inverse_operator(fwd, evoked, alpha='auto')
    else:
        solver_.make_inverse_operator(fwd, alpha='auto')
    stc_hat = solver_.apply_inverse_operator(evoked)
    stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))



,Median,Variance
Bayesian Beamformer,-0.006836,0.085005
Backus-Gilbert,0.040950,0.153385
wMNE,0.097488,0.034515
Bayesian LORETA,0.119697,0.070663
Multiple Sparse Priors,0.119320,0.106239
Bayesian MNE,0.138442,0.100490
eLORETA,0.169089,0.098899
dSPM,0.163615,0.082588
S-MAP,0.217309,0.151754
MNE,0.213278,0.111178


In [ ]:
from mne.minimum_norm import make_inverse_operator as mne_inverse
from mne.minimum_norm import apply_inverse as mne_apply
from mne import make_ad_hoc_cov
noise_cov = make_ad_hoc_cov(evoked.info, verbose=0)
mne_io = mne_inverse(evoked.info, fwd, noise_cov=noise_cov, fixed=True, loose=0, depth=0, verbose=0)
stc_hat = mne_apply(evoked, mne_io, method="MNE", verbose=0)
stc_hat.plot(**pp)